In [ ]:
pip install -U --pre pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.8/105.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.2/77.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 M

In [ ]:
from pycaret.regression import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ProjectData/ph3_1754148.csv')
var_arr = ['field5', 'field6']
tot_arr = ['field1', 'field2', 'field3', 'field4', 'field7', 'field8', 'entry_id']

In [ ]:
def manageFields(field, tot_arr1):
  temp_list = tot_arr1.copy()
  temp_list.remove(field)
  temp_df = df
  temp_df = temp_df.drop(temp_list,axis=1)
  temp_list = tot_arr1
  return temp_df

In [ ]:
df_no2 = manageFields('field1', tot_arr)
df_o3 = manageFields('field2', tot_arr)
df_so2 = manageFields('field3', tot_arr)
df_co = manageFields('field4', tot_arr)
df_pm2_5 = manageFields('field7', tot_arr)
df_pm10 = manageFields('field8', tot_arr)

In [ ]:
def removeGmt(df, field):
  df['created_at'] = df['created_at'].apply(lambda x:x.split('+')[0])
  df['created_at'] = pd.to_datetime(df['created_at'])
  df['created_at'] = df['created_at'].dt.round('min')
  field_ma = field+'_ma'
  df[field_ma] = df[field].rolling(window=5, center=False).mean()
  df = df.dropna()
  df = df[1::15]
  return df

In [ ]:
df_no2 = removeGmt(df_no2, 'field1');
df_o3 = removeGmt(df_o3, 'field2');
df_so2 = removeGmt(df_so2, 'field3');
df_co = removeGmt(df_co, 'field4');
df_pm2_5 = removeGmt(df_pm2_5, 'field7');
df_pm10 = removeGmt(df_pm10, 'field8');

In [ ]:
def createData(df, field):
  df = df.copy()
  df['Month'] = [i.month for i in df['created_at']]
  df['Day'] = [i.day for i in df['created_at']]
  df['Hour'] = [i.hour for i in df['created_at']]
  df['Minute'] = [i.minute for i in df['created_at']]
  df['year'] = [i.year for i in df['created_at']]
  df = df.sort_values(by=['year', 'Month', 'Day', 'Hour', 'Minute'])
  dat = df['created_at']
  df = df.drop(['created_at','year'], axis=1)
  field_ma = field + '_ma'
  df = df[['Month','Day','Hour', 'Minute', 'field5', 'field6', f"{field}", f"{field_ma}"]]
  return [df, dat]

In [ ]:
[df_no2, dat] = createData(df_no2, 'field1')
[df_o3, dat] = createData(df_o3, 'field2')
[df_so2, dat] = createData(df_so2, 'field3')
[df_co, dat] = createData(df_co, 'field4')
[df_pm2_5, dat] = createData(df_pm2_5, 'field7')
[df_pm10, dat] = createData(df_pm10, 'field8')

In [ ]:
def train_test_split(df):
  train_data = train_data = df[0: int(len(df)*0.995)]
  test_data = df[len(train_data): len(df)]
  return [train_data, test_data]

In [ ]:
[train_no2, test_no2] = train_test_split(df_no2)
[train_o3, test_o3] = train_test_split(df_o3)
[train_so2, test_so2] = train_test_split(df_so2)
[train_co, test_co] = train_test_split(df_co)
[train_pm2_5, test_pm2_5] = train_test_split(df_pm2_5)
[train_pm10, test_pm10] = train_test_split(df_pm10)

# NO2 Model


In [ ]:
def makeSetup(train_data, test_data, field):
  field_ma = field+'_ma'
  s = setup(data=train_data, target=field_ma,test_data=test_data,
                        fold_strategy = 'timeseries', numeric_features = ['Month','Day','Hour','Minute','field5','field6'], 
                        fold = 3, transform_target = True, session_id = 123)
  return s

s_no2 = makeSetup(train_no2, test_no2, 'field1')

no2_gbr = create_model('gbr')
tuned_gbr = tune_model(no2_gbr, n_iter=10, custom_grid={'learning_rate':[0.05, 0.06, 0.04], 'max_depth':[3, 4, 5]})
print(tuned_gbr.get_params())
def createPredictions(model, test_data, dat):
  prediction_holdout = predict_model(model)
  predictions = predict_model(model, data=test_data)
  predictions['Date'] = dat
  return predictions

no2_predictions = createPredictions(tuned_gbr, test_no2, dat)

,Description,Value
0,Session id,123
1,Target,field1_ma
2,Target type,Regression
3,Original data shape,"(77480, 8)"
4,Transformed data shape,"(77480, 8)"
5,Transformed train set shape,"(77092, 8)"
6,Transformed test set shape,"(388, 8)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.5920,13.4532,3.6679,0.8853,0.4321,0.3361
1,2.8091,14.4392,3.7999,0.8577,0.4103,0.2977
2,2.6878,14.0759,3.7518,0.8794,0.4071,0.2805
Mean,2.6963,13.9894,3.7398,0.8741,0.4165,0.3048
Std,0.0888,0.4071,0.0546,0.0119,0.0111,0.0232


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.5997,13.6096,3.6891,0.8840,0.4286,0.3289
1,2.7404,13.8538,3.7221,0.8634,0.4055,0.2917
2,2.6773,13.8153,3.7169,0.8817,0.4041,0.2778
Mean,2.6725,13.7596,3.7094,0.8764,0.4127,0.2995
Std,0.0576,0.1072,0.0145,0.0092,0.0112,0.0216


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 9 candidates, totalling 27 fits


{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.06, 'loss': 'squared_error', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'random_state': 123, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,11.8211,210.3252,14.5026,-24.2611,0.9597,1.9716


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,11.8211,210.3252,14.5026,-24.2611,0.9597,1.9716


# O3 Model

In [ ]:
def makeSetup(train_data, test_data, field):
  field_ma = field+'_ma'
  s = setup(data=train_data, target=field_ma,test_data=test_data,
                        fold_strategy = 'timeseries', numeric_features = ['Month','Day','Hour','Minute','field5','field6'], 
                        fold = 3, transform_target = True, session_id = 123)
  return s

s_o3 = makeSetup(train_o3, test_o3, 'field2')

o3_lgbm = create_model('lightgbm')
tuned_lgbm = tune_model(o3_lgbm, n_iter=10, custom_grid={'learning_rate':[0.05, 0.06, 0.04, 0.08], 'max_depth':[3, 4, 5]})

def createPredictions(model, test_data, dat):
  prediction_holdout = predict_model(model)
  predictions = predict_model(model, data=test_data)
  predictions['Date'] = dat
  return predictions

o3_predictions = createPredictions(tuned_lgbm, test_o3, dat)

,Description,Value
0,Session id,123
1,Target,field2_ma
2,Target type,Regression
3,Original data shape,"(77480, 8)"
4,Transformed data shape,"(77480, 8)"
5,Transformed train set shape,"(77092, 8)"
6,Transformed test set shape,"(388, 8)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,5.3162,56.4679,7.5145,0.8489,0.4692,0.4604
1,5.4984,52.5244,7.2474,0.7166,0.4020,0.2591
2,6.4960,88.4429,9.4044,0.7894,0.3502,0.2729
Mean,5.7702,65.8117,8.0554,0.7850,0.4071,0.3308
Std,0.5186,16.0834,0.9601,0.0541,0.0487,0.0918


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.9951,51.5752,7.1816,0.8620,0.4372,0.3957
1,5.1865,47.4196,6.8862,0.7441,0.3937,0.2503
2,5.7605,63.9920,7.9995,0.8476,0.3368,0.2588
Mean,5.3140,54.3290,7.3558,0.8179,0.3893,0.3016
Std,0.3252,7.0403,0.4709,0.0525,0.0411,0.0666


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 10 candidates, totalling 30 fits


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,26.5233,1219.0524,34.9149,-145.3308,1.3274,3.2243


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,26.5233,1219.0524,34.9149,-145.3308,1.3274,3.2243


#SO2 Model

In [ ]:
def makeSetup(train_data, test_data, field):
  field_ma = field+'_ma'
  s = setup(data=train_data, target=field_ma,test_data=test_data,
                        fold_strategy = 'timeseries', numeric_features = ['Month','Day','Hour','Minute','field5','field6'], 
                        fold = 3, transform_target = True, session_id = 123)
  return s

s_so2 = makeSetup(train_so2, test_so2, 'field3')

so2_lgbm = create_model('ada')
tuned_lgbm = tune_model(so2_lgbm, n_iter=10, custom_grid={'learning_rate':[0.05, 0.06, 0.04]})

def createPredictions(model, test_data, dat):
  prediction_holdout = predict_model(model)
  predictions = predict_model(model, data=test_data)
  predictions['Date'] = dat
  return predictions

so2_predictions = createPredictions(tuned_lgbm, test_so2, dat)

,Description,Value
0,Session id,123
1,Target,field3_ma
2,Target type,Regression
3,Original data shape,"(77480, 8)"
4,Transformed data shape,"(77480, 8)"
5,Transformed train set shape,"(77092, 8)"
6,Transformed test set shape,"(388, 8)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,32.5087,2300.3549,47.9620,0.9840,0.7142,0.2532
1,28.5205,1961.7391,44.2915,0.9785,0.6130,0.2695
2,26.4611,1341.3378,36.6243,0.9732,0.6704,0.2730
Mean,29.1634,1867.8106,42.9593,0.9786,0.6659,0.2652
Std,2.5104,397.1107,4.7235,0.0044,0.0415,0.0086


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,32.3181,3879.2346,62.2835,0.9730,0.4826,0.1857
1,26.9733,1519.3723,38.9791,0.9833,0.4944,0.2311
2,29.4011,1706.5291,41.3102,0.9659,0.5545,0.2518
Mean,29.5642,2368.3787,47.5243,0.9741,0.5105,0.2229
Std,2.1850,1071.0653,10.4797,0.0071,0.0315,0.0276


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 3 candidates, totalling 9 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,str,252.9172,89558.7793,299.2637,-2312.1335,2.5368,12.9284


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,str,252.9172,89558.7793,299.2637,-2312.1335,2.5368,12.9284


# CO Model

In [ ]:
def makeSetup(train_data, test_data, field):
  field_ma = field+'_ma'
  s = setup(data=train_data, target=field_ma,test_data=test_data,
                        fold_strategy = 'timeseries', numeric_features = ['Month','Day','Hour','Minute','field5','field6'], 
                        fold = 3, transform_target = True, session_id = 123)
  return s

s_co = makeSetup(train_co, test_co, 'field4')

co_rf = create_model('rf')
# tuned_rf = tune_model(co_rf)

def createPredictions(model, test_data, dat):
  prediction_holdout = predict_model(model)
  predictions = predict_model(model, data=test_data)
  predictions['Date'] = dat
  return predictions

co_predictions = createPredictions(co_rf, test_co, dat)

,Description,Value
0,Session id,123
1,Target,field4_ma
2,Target type,Regression
3,Original data shape,"(77480, 8)"
4,Transformed data shape,"(77480, 8)"
5,Transformed train set shape,"(77092, 8)"
6,Transformed test set shape,"(388, 8)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.0278,10.4378,3.2308,0.3898,0.5381,0.7899
1,0.6107,1.9682,1.4029,0.0496,0.4937,0.8480
2,0.2212,1.5804,1.2571,0.2714,0.2896,0.8092
Mean,0.6199,4.6621,1.9636,0.2369,0.4405,0.8157
Std,0.3294,4.0871,0.8980,0.1410,0.1082,0.0242


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.3127,1.5735,1.2544,-134.6770,0.3506,5.3301


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.3127,1.5735,1.2544,-134.6770,0.3506,5.3301


#PM2.5 Model

In [ ]:
def makeSetup(train_data, test_data, field):
  field_ma = field+'_ma'
  s = setup(data=train_data, target=field_ma,test_data=test_data,
                        fold_strategy = 'timeseries', numeric_features = ['Month','Day','Hour','Minute','field5','field6'], 
                        fold = 3, transform_target = True, session_id = 123)
  return s

s_pm2_5 = makeSetup(train_pm2_5, test_pm2_5, 'field7')

pm2_5_lgbm = create_model('gbr')
tuned_lgbm = tune_model(pm2_5_lgbm)

def createPredictions(model, test_data, dat):
  prediction_holdout = predict_model(model)
  predictions = predict_model(model, data=test_data)
  predictions['Date'] = dat
  return predictions

pm2_5_predictions = createPredictions(tuned_lgbm, test_pm2_5, dat)

,Description,Value
0,Session id,123
1,Target,field7_ma
2,Target type,Regression
3,Original data shape,"(77331, 8)"
4,Transformed data shape,"(77331, 8)"
5,Transformed train set shape,"(76944, 8)"
6,Transformed test set shape,"(387, 8)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.2584,2.9941,1.7304,0.9881,0.1580,0.1349
1,1.0467,3.2079,1.7911,0.9856,0.1438,0.1494
2,1.1468,2.5139,1.5855,0.9894,0.0988,0.0818
Mean,1.1506,2.9053,1.7023,0.9877,0.1336,0.1220
Std,0.0865,0.2902,0.0862,0.0016,0.0253,0.0291


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.3307,3.3316,1.8253,0.9867,0.1501,0.1282
1,1.2539,4.4309,2.1050,0.9800,0.2145,0.2563
2,1.3096,3.3142,1.8205,0.9860,0.1336,0.1162
Mean,1.2981,3.6923,1.9169,0.9843,0.1661,0.1669
Std,0.0324,0.5224,0.1330,0.0030,0.0349,0.0634


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 10 candidates, totalling 30 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,23.6581,658.1380,25.6542,-200.6093,1.2471,2.7974


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,23.6581,658.1380,25.6542,-200.6093,1.2471,2.7974


#PM10 Model

In [ ]:
def makeSetup(train_data, test_data, field):
  field_ma = field+'_ma'
  s = setup(data=train_data, target=field_ma,test_data=test_data,
                        fold_strategy = 'timeseries', numeric_features = ['Month','Day','Hour','Minute','field5','field6'], 
                        fold = 3, transform_target = True, session_id = 123)
  return s

s_pm10 = makeSetup(train_pm10, test_pm10, 'field8')

pm10_lgbm = create_model('gbr')
tuned_lgbm = tune_model(pm10_lgbm)

def createPredictions(model, test_data, dat):
  prediction_holdout = predict_model(model)
  predictions = predict_model(model, data=test_data)
  predictions['Date'] = dat
  return predictions

pm10_predictions = createPredictions(tuned_lgbm, test_pm10, dat)

,Description,Value
0,Session id,123
1,Target,field8_ma
2,Target type,Regression
3,Original data shape,"(77480, 8)"
4,Transformed data shape,"(77480, 8)"
5,Transformed train set shape,"(77092, 8)"
6,Transformed test set shape,"(388, 8)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.6500,4.9738,2.2302,0.9865,0.1202,0.1055
1,1.4981,4.6201,2.1494,0.9859,0.1379,0.1367
2,1.7535,5.6217,2.3710,0.9840,0.1152,0.0987
Mean,1.6338,5.0719,2.2502,0.9855,0.1244,0.1136
Std,0.1049,0.4148,0.0916,0.0011,0.0098,0.0166


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.8497,6.1762,2.4852,0.9832,0.1314,0.1162
1,1.8692,7.0912,2.6629,0.9784,0.1863,0.2007
2,1.9448,6.9104,2.6288,0.9803,0.1316,0.1142
Mean,1.8879,6.7259,2.5923,0.9806,0.1498,0.1437
Std,0.0410,0.3957,0.0770,0.0020,0.0258,0.0403


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 10 candidates, totalling 30 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,29.8563,1083.2598,32.9129,-379.7361,1.4691,3.8400


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,29.8563,1083.2598,32.9129,-379.7361,1.4691,3.8400


# Predictions

In [ ]:
import plotly.express as px

In [ ]:
def createChart(predictions, field_ma, title):
  temp_df = predictions.rename({field_ma:'Actual', 'prediction_label': 'Predicted'}, axis=1, inplace=False)
  fig1 = px.line(temp_df, x='Date', y=['Actual', 'Predicted']).update_layout(xaxis_title="Date and time", yaxis_title=title)
  fig1.show()
  fig2 = px.scatter(temp_df, x='Predicted', y='Actual')
  fig2.show()

In [ ]:
createChart(no2_predictions, 'field1_ma', 'NO2')

In [ ]:
createChart(o3_predictions, 'field2_ma', 'O3')

In [ ]:
createChart(so2_predictions, 'field3_ma', 'SO2')

In [ ]:
createChart(co_predictions, 'field4_ma', 'CO')

In [ ]:
createChart(pm2_5_predictions, 'field7_ma', 'PM2.5')

In [ ]:
createChart(pm10_predictions, 'field8_ma', 'PM10')